# Moby Eye Tracking

Notebook for developing fast, accurate eye tracking straight from your webcam.

In [2]:
# Imports
import os
import face_recognition
import cv2
import random
import time
import sys

import matplotlib.pyplot as plt 
import numpy as np

from tkinter import *

from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler

import keras 
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, Dense, MaxPool2D, Flatten 

%matplotlib inline 

Using TensorFlow backend.


In [3]:
# Convenience functions
def small_dot(tkinter_canvas, centre_x, centre_y, radius=5, fill="red"):
    """Given the centre point of a dot, this convenience function will draw a small dot with given radius"""
    
    tkinter_canvas.create_oval(centre_x - radius, centre_y - radius,
                               centre_x + radius, centre_y + radius, fill=fill)
    
    return

def random_dot(tkinter_canvas, tk_width, tk_height):
    
    border = 5 # Should be same, or higher than radius of dots
    
    random_width = random.randint(border, tk_width - border)
    random_height = random.randint(border, tk_height - border)
    
    small_dot(tkinter_canvas, random_width, random_height)
    
    return random_width, random_height

def neural_model(dummy_sample):
    
    print("About to initialise a neural network with input shape: ", dummy_sample.shape)
    
    visible = Input(shape=(dummy_sample.shape))
    
    c11 = Conv2D(8, 3)(visible)
    c12 = Conv2D(8, 3)(c11)
    p1 = Conv2D(16, 1, strides=2)(c12)
    c21 = Conv2D(16, 3)(p1)
    c22 = Conv2D(16, 3)(c21)
    p2 = Conv2D(32, 1, strides=2)(c22)
    #c31 = Conv2D(8, 3)(p2)
    #c32 = Conv2D(8, 3)(c31)
    #p3 = Conv2D(16, 1, strides=2)(c32)
    
    f1 = Flatten()(p2)
    d1 = Dense(50, activation="relu")(f1)
    d2 = Dense(50, activation="relu")(d1)
    output = Dense(2)(d2)
    
    model = Model(inputs=visible, outputs=output)
    
    model.compile(loss=keras.losses.MeanSquaredError(), optimizer="adam")
    
    return model

def extract_facial_features(frame, display=False):
    
    # Basic code for facial landmark extraction from webcam from:
    # https://elbruno.com/2019/05/29/vscode-lets-do-some-facerecognition-with-20-lines-in-python-3-n/    
    rgb_frame = frame[:, :, ::-1].copy()
    frame_copy = frame.copy()
    bw_frame = np.mean(rgb_frame, axis=2)

    face_landmarks_list = face_recognition.face_landmarks(rgb_frame)
    
    # Extract region around eyes, before green lines added. Uses face_recognition
    border_height = 10
    border_width = 15
    
    # Creat linear ingredients to bundle with the eye data
    grad_x = np.zeros(frame_copy.shape[:2], dtype=np.float)
    grad_y = np.zeros(frame_copy.shape[:2], dtype=np.float)
    
    for i in range(border_height * 2):
        grad_x[i, :] = i / (border_height * 2)
        
    for j in range(border_width * 2):
        grad_y[:, j] = j / (border_width * 2)
    
    try:
        left_eye = np.mean(np.array(face_landmarks_list[0]["left_eye"]), axis=0, dtype=int)
        left_eye_region = bw_frame[left_eye[1] - border_height: left_eye[1] + border_height,
                                   left_eye[0] - border_width: left_eye[0] + border_width]
        left_eye_x_grad = grad_x[left_eye[1] - border_height: left_eye[1] + border_height,
                                 left_eye[0] - border_width: left_eye[0] + border_width]
        left_eye_y_grad = grad_y[left_eye[1] - border_height: left_eye[1] + border_height,
                                 left_eye[0] - border_width: left_eye[0] + border_width]
        
        left_eye_flattened = left_eye_region.reshape(1,-1)[0]
    
        right_eye = np.mean(np.array(face_landmarks_list[0]["right_eye"]), axis=0, dtype=int)
        right_eye_region = bw_frame[right_eye[1] - border_height: right_eye[1] + border_height,
                                    right_eye[0] - border_width: right_eye[0] + border_width]
        right_eye_x_grad = grad_x[right_eye[1] - border_height: right_eye[1] + border_height,
                                  right_eye[0] - border_width: right_eye[0] + border_width]
        right_eye_y_grad = grad_y[right_eye[1] - border_height: right_eye[1] + border_height,
                                  right_eye[0] - border_width: right_eye[0] + border_width]
        
        right_eye_flattened = right_eye_region.reshape(1,-1)[0]
            
        # Scale features
        scaler = StandardScaler()
        left_eye_region = scaler.fit_transform(left_eye_region)
        right_eye_region = scaler.fit_transform(right_eye_region)
        
        eyes_and_gradients = np.stack((left_eye_region, left_eye_x_grad, left_eye_y_grad,
                                       right_eye_region, right_eye_x_grad, right_eye_y_grad), axis=2)
    except IndexError:
        print("Could not extract eye regions, probably because face not detected")
        return [], [], [], []
        
    for face_landmarks in face_landmarks_list:

        for facial_feature in face_landmarks.keys():
            pts = np.array([face_landmarks[facial_feature]], np.int32) 
            pts = pts.reshape((-1,1,2))
            cv2.polylines(frame, [pts], False, (0,255,0))

    if display:
        cv2.imshow('Video', frame)
        
    # print(face_landmarks_list)
    
    # I suspect this code will break if multiple faces
    landmark_array = np.array(np.zeros((0, 2)))
    if face_landmarks_list != []:
        for landmark in face_landmarks_list[0].values():
            landmark_array = np.concatenate((landmark_array, np.array(landmark)))
    else:
        print("No face detected") 
    
    # Concatenate the extracted facial features, with the region around the eyes 
    everything_array = np.concatenate(
        (landmark_array[0], left_eye_flattened, right_eye_flattened))
    landmark_array = landmark_array[0]
    
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break
    
    everything_array = everything_array.reshape(1, -1)
    landmark_array = landmark_array.reshape(1, -1)
    
    # print(landmark_array[0].shape)
    
    return rgb_frame, everything_array, landmark_array, eyes_and_gradients

def predict_gaze(video_capture, webcam_resolution,  
                 tk_width, tk_height, model, model_type, canvas):
    
    ret, frame = video_capture.read()
    (rgb_frame, everything_array, 
     landmark_array, eyes_and_gradients) = extract_facial_features(frame)
    
    try:
        if model_type == "neural net":
            X = np.expand_dims(eyes_and_gradients, 0)
            predicted_gaze = model.predict(X)[0]
        else:
            predicted_gaze = model.predict(everything_array)[0]
    
        print("Predicted gaze is: ", predicted_gaze)
    except ValueError:
        print("Could not predict, probably no face in image")
        predicted_gaze = np.array([0., 0.])
    
    # Scale the prediction to webcam resolution
    predicted_pixel = [predicted_gaze[0] * tk_width, predicted_gaze[1] * tk_height]
    # print(predicted_pixel, predicted_gaze, webcam_resolution)
    
    # Display the prediction as a grey circle
    small_dot(canvas, predicted_pixel[0], predicted_pixel[1], radius=5, fill="grey")
    
    return rgb_frame, everything_array, eyes_and_gradients, predicted_gaze

def capture(counter, canvas, model, model_type, training_X, training_y, tk_width, tk_height, 
            video_capture, rgb_frame, webcam_resolution, 
            landmark_array, eyes_and_gradients, current_target, predicted_gaze, move_smoothly=False, randomise_dot=True):
    """Will capture an image, coordinate pair when the user is looking at the dot"""
    
    path = "data/MZeina_2/"
    train_every = 1
        
    # print("About to learn...")
    if len(landmark_array) != 0:
        current_target = np.array(current_target) / np.array([tk_width, tk_height])
        
        if model_type == "neural net":
            # Neural network can train on each sample at a time, unlike random forest
            training_X = np.expand_dims(eyes_and_gradients, 0)
            training_y = np.expand_dims(current_target, 0)
            # training_X.append(eyes_and_gradients)
        else:
            training_X.append(landmark_array[0])
            training_y.append(current_target)
        
        plt.imsave(path + str(current_target) + ".jpg", rgb_frame)
        
        if counter % train_every == 0:
            model.fit(training_X, training_y)
        
    else:
        print("Face not detected, will not train on this sample")
    
    #canvas.delete("all")
    if move_smoothly:
        speed = 20
        scaled_counter = (counter * speed) % (tk_width * tk_height)
        target_x = (scaled_counter // tk_height * speed) % tk_width
        if (scaled_counter // tk_height)%2 == 0:
            target_y = scaled_counter % tk_height
        else:
            # reverse the direction for alternative lines, so it doesn't skip up to the top
            target_y = tk_height - scaled_counter % tk_height
        print("counter, scaled_counter, are :", counter, scaled_counter)
        print("about to move small circle to", target_x, target_y)
        small_dot(canvas, target_x, target_y)
        current_target = [target_x, target_y]
    elif randomise_dot:
        current_target = random_dot(canvas, tk_width, tk_height)
    # print(random_width, random_height)
    
    return model, current_target

def train_retrospectively(path_to_images, model):
    
    # Build data frame of past images, and the extract features
    # For any non-small neural network, I should replace this technique with a generator
    
    training_X = []
    training_y = []
    counter = 0
    path_to_images = "captures_one/"
    
    # Currently only looks in a single directory
    files = os.listdir(path_to_images)
    
    for file in files:
        print("About to process image number ", counter)
        image = cv2.imread(path_to_images + file)
        rgb_frame, everything_array, landmark_array, eyes_and_gradients = extract_facial_features(image)
        coordinates = [float(coordinate) for coordinate in file[1: -5].split(" ") if len(coordinate) != 0]
        
        training_X.append(eyes_and_gradients)
        training_y.append(coordinates)
        
        counter += 1
                       
    return training_X, training_y

In [4]:
# Functions that leverage the above to do something useful
def train_and_preview(pretrained_model=None):
    ########## Universal Initialisation ##########
    counter = 0
    captures_per_point = 5
    
    ########## Initialise Video Stream ##########
    video_capture = cv2.VideoCapture(0)
    
    # Extract webcam resolution
    ret, frame = video_capture.read()
    webcam_resolution = frame.shape[:2]
    # print(webcam_resolution) 
    
    ########## Initialise ML Model ##########
    
    # Dummy sample, to help initialising models
    (rgb_frame, dummy_features, 
     landmark_array, eyes_and_gradients) = extract_facial_features(frame)
    
    model_type = "neural net"
    
    if pretrained_model:
        model = pretrained_model
    elif model_type == "random forest":
        # Random forest 
        RF = RandomForestRegressor(n_estimators=500, n_jobs=-1, warm_start=False)
        model = MultiOutputRegressor(RF)
        model.fit(np.zeros_like(dummy_features), np.array([0.5, 0.5]).reshape(1, -1))
    elif model_type == "neural net":
        model = neural_model(eyes_and_gradients)
        model.summary()
        
    # To do:Train on existing pictures
    
    # Initialise
    training_X = []
    training_y = []
    
    ########## Initialise Tkinter ##########
    window = Tk()
    window.attributes("-fullscreen", True)
    
    window.update_idletasks() 
    tk_width = window.winfo_width() 
    tk_height = window.winfo_height()

    canvas = Canvas(window, width = tk_width, height = tk_height)
    canvas.pack()
    
    window.bind("<F11>", lambda event: window.attributes("-fullscreen",
                                        not window.attributes("-fullscreen")))
    window.bind("<Escape>", lambda event: window.attributes("-fullscreen", False))
    # window.bind("c", lambda event: capture(canvas, RFMO, tk_width, tk_height, video_capture, webcam_resolution, landmark_array, current_target, predicted_gaze))
    
    # Variables to store red dot target
    current_target = random_dot(canvas, tk_width, tk_height)
    
    while True:
        
        rgb_frame, landmark_array, eyes_and_gradients, predicted_gaze = predict_gaze(
            video_capture, webcam_resolution, tk_width, tk_height, model, model_type, canvas)
        
        if counter % 4 == 0 and counter != 0:
            canvas.delete("all")
            
            RFMO, current_target = capture(
                counter, canvas, model, model_type, training_X, training_y, tk_width, tk_height, video_capture, 
                rgb_frame, webcam_resolution, landmark_array, eyes_and_gradients, 
                current_target, predicted_gaze, randomise_dot=True)
                
        counter += 1
        
        # Update GUI
        window.update_idletasks()
        window.update()
    return

In [ ]:
train_and_preview()

About to initialise a neural network with input shape:  (20, 30, 6)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20, 30, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 28, 8)         440       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 26, 8)         584       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 13, 16)         144       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 11, 16)         2320      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 9, 16)          2320      
_________________________________________________________

Predicted gaze is:  [0.5689971 0.9262949]
Predicted gaze is:  [0.38578942 1.000396  ]
Predicted gaze is:  [0.46121842 0.96889424]
Predicted gaze is:  [0.5440456  0.94472843]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0230
Predicted gaze is:  [0.6446771  0.98478925]
Predicted gaze is:  [0.6822742 0.8560683]
Predicted gaze is:  [0.4869808  0.25140485]
Predicted gaze is:  [0.5297472  0.20407653]
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0080
Predicted gaze is:  [0.373885   0.07130861]
Predicted gaze is:  [0.5327987  0.21400893]
Predicted gaze is:  [0.73182505 0.7554438 ]
Predicted gaze is:  [0.9123974  0.56790364]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0243
Predicted gaze is:  [0.7355788 0.5547425]
Predicted gaze is:  [0.617337  0.5347633]
Predicted gaze is:  [0.1725275  0.25024053]
Predicted gaze is:  [0.2470136  0.21143769]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0185


Predicted gaze is:  [0.3188148 0.3066251]
Predicted gaze is:  [0.5263253  0.30851257]
Predicted gaze is:  [0.5931783  0.47015882]
Predicted gaze is:  [0.56662405 0.47023875]
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0780
Predicted gaze is:  [0.41106805 0.4051017 ]
Predicted gaze is:  [0.6169406 0.38848  ]
Predicted gaze is:  [0.5452147  0.18326382]
Predicted gaze is:  [0.5413896 0.3062604]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0207
Predicted gaze is:  [0.5474892  0.08645494]
Predicted gaze is:  [0.40397197 0.33594835]
Predicted gaze is:  [0.2806434  0.33635864]
Predicted gaze is:  [0.37729052 0.28744423]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.1523
Predicted gaze is:  [0.28062105 0.5213406 ]
Predicted gaze is:  [0.31318688 0.4769073 ]
Predicted gaze is:  [0.2594661  0.46802258]
Predicted gaze is:  [0.53872913 0.48159978]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.030

Predicted gaze is:  [0.18088244 0.4159868 ]
Predicted gaze is:  [0.5266168 0.6556841]
Predicted gaze is:  [0.6227251  0.25908157]
Predicted gaze is:  [0.5891365  0.24226308]
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0515
Predicted gaze is:  [0.49636698 0.20135027]
Predicted gaze is:  [0.49570203 0.29590175]
Predicted gaze is:  [1.015562  0.6547794]
Predicted gaze is:  [1.030487   0.72743136]
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0411
Predicted gaze is:  [1.0204071  0.83342934]
Predicted gaze is:  [0.55618906 0.7641865 ]
Predicted gaze is:  [-0.00462333  0.26989934]
Predicted gaze is:  [0.12933311 0.41306752]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0082
Predicted gaze is:  [0.15242592 0.35076082]
Predicted gaze is:  [0.40692824 0.27951875]
Predicted gaze is:  [0.28271365 0.20283589]
Predicted gaze is:  [0.42056906 0.17223993]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0

Predicted gaze is:  [0.59714633 0.22279768]
Predicted gaze is:  [0.4820359  0.24510425]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0116
Predicted gaze is:  [0.51339746 0.22865105]
Predicted gaze is:  [0.16888371 0.28009105]
Predicted gaze is:  [0.08239368 0.29351136]
Predicted gaze is:  [0.00454372 0.22520952]
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0115
Predicted gaze is:  [-0.0385134   0.22016133]
Predicted gaze is:  [0.05643969 0.18327446]
Predicted gaze is:  [0.45513758 0.33728746]
Predicted gaze is:  [0.41258028 0.39164093]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0197
Predicted gaze is:  [0.4628594  0.54090035]
Predicted gaze is:  [0.3854779 0.5115251]
Predicted gaze is:  [0.6945639  0.22218496]
Predicted gaze is:  [0.5196542  0.21945624]
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.0274
Predicted gaze is:  [0.5718049  0.19457722]
Predicted gaze is:  [0.58102024 0.16

...

Predicted gaze is:  [0.488507  0.7246684]
Predicted gaze is:  [0.05429199 0.75834346]
Predicted gaze is:  [0.1751746 0.5068482]
Predicted gaze is:  [0.10285562 0.5344752 ]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0076
Predicted gaze is:  [0.16392863 0.58114123]
Predicted gaze is:  [0.42175952 0.2678925 ]
Predicted gaze is:  [0.5546379  0.17537564]
Predicted gaze is:  [0.40346372 0.23998702]
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0081
Predicted gaze is:  [0.4700899  0.19429272]
Predicted gaze is:  [-0.01202798  0.9160991 ]
Predicted gaze is:  [0.07135034 0.8853661 ]
Predicted gaze is:  [0.17145032 0.6817381 ]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0117
Predicted gaze is:  [0.07229762 0.7502471 ]
Predicted gaze is:  [0.00309129 0.73670703]
Predicted gaze is:  [0.03462997 0.6534951 ]
Predicted gaze is:  [0.10174068 0.48690182]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 8

Predicted gaze is:  [0.63856393 0.71572   ]
Predicted gaze is:  [0.18746927 0.104547  ]
Predicted gaze is:  [0.33373514 0.93147504]
Predicted gaze is:  [0.23207965 0.04067514]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0200
Predicted gaze is:  [0.081067   0.14358379]
Predicted gaze is:  [0.35174257 0.599787  ]
Predicted gaze is:  [0.24357715 0.71843606]
Predicted gaze is:  [0.2545123 0.7370833]
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0101
Predicted gaze is:  [0.25618762 0.6498742 ]
Predicted gaze is:  [0.48483604 0.62512   ]
Predicted gaze is:  [0.6002933 0.6733715]
Predicted gaze is:  [0.48255128 0.6330335 ]
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0148
Predicted gaze is:  [0.52370393 0.597102  ]
Predicted gaze is:  [0.49652317 0.42887947]
Predicted gaze is:  [0.44424072 0.4382044 ]
Predicted gaze is:  [0.571692   0.42084298]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0

Predicted gaze is:  [0.3826155  0.13351555]
Predicted gaze is:  [0.2274414  0.25798953]
Predicted gaze is:  [0.26702824 0.4245258 ]
Predicted gaze is:  [0.29705536 0.3135169 ]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0182
Predicted gaze is:  [0.1987953  0.39433807]
Predicted gaze is:  [0.8561466  0.41556293]
Predicted gaze is:  [0.75333023 0.31404123]
Predicted gaze is:  [0.21003067 0.5922835 ]
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.2388
Predicted gaze is:  [0.48492765 0.2604528 ]
Predicted gaze is:  [0.8118132  0.31773892]
Predicted gaze is:  [0.57710576 0.24810308]
Predicted gaze is:  [0.77919316 0.15912604]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0054
Predicted gaze is:  [0.7521741  0.22214141]
Predicted gaze is:  [0.4162314 0.431924 ]
Predicted gaze is:  [0.6713232  0.63855094]
Predicted gaze is:  [0.58003765 0.5872815 ]
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0

Predicted gaze is:  [0.6880685  0.48609012]
Predicted gaze is:  [0.57412004 0.08078611]
Predicted gaze is:  [0.14040278 0.15617676]
Predicted gaze is:  [0.3181736  0.18036516]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0206
Predicted gaze is:  [0.4249611 0.1376257]
Predicted gaze is:  [0.4407519  0.48888665]
Predicted gaze is:  [0.2606821  0.23364303]
Predicted gaze is:  [0.13549298 0.2569404 ]
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0190
Predicted gaze is:  [0.24333388 0.2252423 ]
Predicted gaze is:  [0.43695095 0.4900606 ]
Predicted gaze is:  [0.38540578 0.41556564]
Predicted gaze is:  [0.9241168  0.18028308]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0045
Predicted gaze is:  [0.6421624 0.3499248]
Predicted gaze is:  [0.2457643  0.38982245]
Predicted gaze is:  [0.5475917  0.39743954]
Predicted gaze is:  [0.29342967 0.4085646 ]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0

Predicted gaze is:  [0.5679825 0.9245652]
Predicted gaze is:  [0.24295667 0.6430977 ]
Predicted gaze is:  [0.2568445 0.6065857]
Predicted gaze is:  [0.34883168 0.64120674]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0172
Predicted gaze is:  [0.24874242 0.7395838 ]
Predicted gaze is:  [0.5659117  0.52621925]
Predicted gaze is:  [0.5251494  0.55509865]
Predicted gaze is:  [0.51358104 0.5538587 ]
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: 0.0031
Predicted gaze is:  [0.56264466 0.58920246]
Predicted gaze is:  [0.20277163 0.24899974]
Predicted gaze is:  [0.21567965 0.19988626]
Predicted gaze is:  [0.20842156 0.2605493 ]
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.0272
Predicted gaze is:  [0.17343113 0.28730893]
Predicted gaze is:  [0.9089756 0.4043479]
Predicted gaze is:  [0.8913749 0.3212844]
Predicted gaze is:  [0.9603695  0.31422168]
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.0065


In [ ]:
video_capture = cv2.VideoCapture(0)
RF = RandomForestRegressor(n_estimators=100, n_jobs=-1, warm_start=True)
RFMO = MultiOutputRegressor(RF)
RFMO.fit(np.zeros_like(extract_facial_features(video_capture)), np.array([0, 0]).reshape(1, -1))
# RFMO.predict(np.array([1,1,1]).reshape(1, -1))
# RFMO.fit(np.array([1,1,1]).reshape(1, -1), np.array([1, 0]).reshape(1, -1))

In [69]:
class ScreenshotGenerator(keras.utils.Sequence):
    
    def __init__(self, path_to_images, batch_size=4):
        
        self.path_to_images = path_to_images
        self.batch_size = batch_size
    
        self.files = []# os.listdir(path_to_images)
        self.filenames = []
        
        for root, dirs, files in os.walk(path_to_images):
            for name in files:
                self.files.append(os.path.join(root, name))
                self.filenames.append(name)
    
    def __len__(self):
        
        return len(self.files) // self.batch_size
    
    def __load__(self, index):
        """Returns and processes a single sample, in conjunction with __getitem__"""
        
        # Ensures that if an image is picked without a succesfully detected face, 
        #  it looks for another random one to replace it
        got_good_image = False
        
        while not got_good_image:
        
            file = self.files[index]
            filename = self.filenames[index]
                        
            image = cv2.imread(file)
            
            rgb_frame, everything_array, landmark_array, eyes_and_gradients = extract_facial_features(image)
            coordinates = [float(coordinate) for coordinate in filename[1: -5].split(" ") if len(coordinate) != 0]
            
            X = eyes_and_gradients
            y = coordinates
            
            if len(X) == 0:
                print("This image did not have a recognisable face, will pull a random one in its place")
                index = random.randint(0, self.__len__())
            else:
                got_good_image = True
        
        return X, y
    
    def __getitem__(self, batch):
        
        batch_X = [self.__load__(index)[0] for index in 
                   range((batch * self.batch_size), (batch + 1) * self.batch_size)]
        batch_y = [self.__load__(index)[1] for index in 
                   range((batch * self.batch_size), (batch + 1) * self.batch_size)]
        
        batch_X = np.array(batch_X)
        batch_y = np.array(batch_y)
        
        return batch_X, batch_y

    def on_epoch_end(self):
        
        return

In [39]:
video_capture = cv2.VideoCapture(0)
_, frame = video_capture.read()
video_capture.release()

In [40]:
(rgb_frame, dummy_features, 
     landmark_array, eyes_and_gradients) = extract_facial_features(frame)

In [41]:
model = neural_model(eyes_and_gradients)
#model.summary()

About to initialise a neural network with input shape:  (20, 30, 6)


In [71]:
screenshot_generator = ScreenshotGenerator("eye_tracking_data/", 4)
model.fit_generator(screenshot_generator, epochs=4)

Epoch 1/4
 860/5687 [===>..........................] - ETA: 2:57:40 - loss: 0.0200

KeyboardInterrupt: 